In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/16 16:21:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/Comment')

In [4]:
sample_data = data.select('value').take(1)

In [5]:
sample_data = sample_data[0].value

In [6]:
data_sample = loads(sample_data)

In [7]:
schema = StructType([ 
    StructField("id",IntegerType(),True), 
    StructField("content",StringType(),True), 
    StructField("thank_count",IntegerType(),True), 
    StructField("customer_id",IntegerType(),True),
    StructField("rating",IntegerType(),True), 
    StructField("created_by",MapType(StringType(),StringType()),True),
    StructField("spid",IntegerType(),True),
    StructField("seller",MapType(StringType(),StringType()),True),
    StructField('product_id', IntegerType(),True),
])

In [8]:
df = data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")

In [9]:
df.createOrReplaceTempView('Comment')

In [10]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 
def cleanText(str_raw):
    # remove tags html
    str_raw = re.sub(CLEANR, ' ', str_raw)

    # remove special character
    str_raw = re.sub('\W+', ' ', str_raw)
    
    # remove number
    str_raw = re.sub("[0-9]+", "", str_raw)
    
    # remove space
    cleantext = re.sub(" +", " ", str_raw)
    return cleantext.lower()

In [11]:
spark.udf.register("cleanText", cleanText,StringType())

<function __main__.cleanText(str_raw)>

In [12]:
df_clean = spark.sql("""
    select id,content contentRaw,cleanText(content) content,thank_count,customer_id,rating,spid,
    seller.id seller_id,product_id, created_by.full_name customer_full_name,created_by.purchased_at purchased_at,
    from_unixtime(CAST(created_by.purchased_at as BIGINT), 'yyyy-MM-dd') date_purchased_at,
    case
        when rating >= 4 then 'Positive'
        when rating = 3 then 'Neutral'
        when rating <3 then 'Negative'
    end as sentiment,
    case
        when rating >= 4 then 2
        when rating = 3 then 1
        when rating <3 then 0
    end as lable
    from Comment c
""")

In [13]:
df_clean.write.mode('append').parquet('hdfs://namenode:9000/TikiCleaned/Comment')

In [14]:
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')

In [15]:
df_comment.first()

Row(id=13707516, contentRaw='Thấy mọi người khen nó quá nhiều nên lên đây chê 1 tí\n- Thứ nhất là chất âm, do cơ bản nó ko thiên về dải nào nên nghe cứ bằng bằng ko ấn tượng gì, bạn nên cài app chỉnh EQ của nó, nâng tầm trải nghiệm ngay lập tức (EQ preset Bright khá tốt)\n- Nhét vào tai hơi khó, cơ bản phải kín mới đạt được chống ồn tốt, mình nhét cũng phải xoay đi xoay lại vài vòng mới khít, đấy là tai mình to còn vừa, còn tai các bạn nữ nhỏ xinh như bạn gái mình là chịu luôn, quá to. Nghe mọi người bảo nên mua thêm tip spinfit 360, chắc sẽ thử xem\n- Các tính năng phụ như tự động chuyển đổi noise cancellling, speak-to-chat dùng mình thấy phiền nhiều hơn có ích, nên tắt để tiết kiệm pin\nCòn lại hoàn hảo :)))', content='thấy mọi người khen nó quá nhiều nên lên đây chê tí thứ nhất là chất âm do cơ bản nó ko thiên về dải nào nên nghe cứ bằng bằng ko ấn tượng gì bạn nên cài app chỉnh eq của nó nâng tầm trải nghiệm ngay lập tức eq preset bright khá tốt nhét vào tai hơi khó cơ bản phải kín